**Connect Google Drive Files**

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Requirements

Install spesific requirements(mmcv,LRP-Error Libraries,WBF Libraries)

In [ ]:
!pip install mmcv

In [ ]:
!pip install mmdet==2.12.0

In [ ]:
!pip install mmcv-full==1.3.4

In [ ]:
!pip install ensemble-boxes

In [ ]:
!pip install git+https://github.com/kemaloksuz/LRP-Error.git#subdirectory=pycocotools
!pip install git+https://github.com/kemaloksuz/LRP-Error.git#subdirectory=panopticapi
!pip install git+https://github.com/kemaloksuz/LRP-Error.git#subdirectory=lvis-api

**Class name will be change in these directiroies**

---



In [ ]:
/usr/local/lib/python3.7/dist-packages/mmdet/datasets/coco.py
/usr/local/lib/python3.7/dist-packages/mmdet/core/evaluation/class_names.py

**Import Libraries**

---



# **Run Preprocessing**

Apply preprocess operations on new images for test with our model

In [ ]:
%cd /content/drive/MyDrive/Makale_2

In [ ]:
from utils.voc2json import voc2json
from utils.preprocess import custom_Dataset

In [ ]:
pre_process=custom_Dataset(source_dir="./source/", target_dir="./test_images/", resize_dim=1000)

In [ ]:
pre_process.runPreProcess()

# **Labeling**

Use this web-site for coco format labelling 

In [ ]:
https://imglab.in/

# Convert XML Labels to Json Format for MMdetection Usage

Run these codes if you have xml label format.

In [ ]:
xml2json=voc2json("/content/drive/MyDrive/Makale_2/xml",'ground_truth')

In [ ]:
xml2json.run()

# Read Models Json For Usage All Models These Are Used In Ensemble Method

Read models json file for get models configs and weights to evaulate on dataset

In [ ]:
%cd /content/drive/MyDrive/Makale_2

In [ ]:
import json

with open('./models.json') as data_file:    
    _models = json.load(data_file)

In [ ]:
for _model in _models['model']:
    a='"'+"jsonfile_prefix="+"./json_output/"+_model['name']+'"'
    !python ./tools/test.py {_model['config_path']} {_model['weight_path']} --format-only --options {a}

# Convert JSON outputs that output of evaluation, to CSV for WBF usage.

Use _mmdetection2wbf for convert json to csv files to WBF usage

In [ ]:
%cd /content/drive/MyDrive/Makale_2

In [ ]:
from utils._mmdetection2wbf import *

In [ ]:
mmdet2wbf=mmdetection2wbf(gt_path="./ground_truth/ground_truth.json", json_path="./json_output",csv_path="./csv_output", init=False)

In [ ]:
mmdet2wbf.main()

In [ ]:
from utils.wbf import wbf

Read ensemble_models json file for get ensemble configuration.(sub network and their weights for usage fusion operation)

In [ ]:
wbf=wbf(gt_path="./ground_truth/ground_truth.json")

In [ ]:
import json

with open('./ensemble_models.json') as data_file:    
    _ensemble_models = json.load(data_file)

In [ ]:
for _model in _ensemble_models['ensemble_models']:
    wbf.predict(_model['submodels'],_model['weights'],_model['name'])